In [2]:
import os
from google.cloud import vision

# Set Google Cloud credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "sunny-furnace-450323-j7-fb55f0c1398a.json"

def detect_image_features(path):
    """Detects labels, objects, logos, text, dominant colors, and faces."""
    client = vision.ImageAnnotatorClient()

    with open(path, "rb") as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    # Label Detection
    label_response = client.label_detection(image=image)
    labels = label_response.label_annotations
    print("Detected Labels:")
    for label in labels:
        print(f"Label: {label.description} (Confidence: {label.score:.2f})")

    # Object Detection
    object_response = client.object_localization(image=image)
    objects = object_response.localized_object_annotations
    print(f"\nDetected Objects:")
    for obj in objects:
        print(f"Object: {obj.name} (Confidence: {obj.score:.2f})")

    # Logo Detection (Brand Extraction)
    logo_response = client.logo_detection(image=image)
    logos = logo_response.logo_annotations
    print(f"\nDetected Logos:")
    if logos:
        for logo in logos:
            print(f"Logo: {logo.description} (Confidence: {logo.score:.2f})")
    else:
        print("No logos detected.")

    # OCR - Text Detection (Brand/Model Extraction)
    text_response = client.text_detection(image=image)
    texts = text_response.text_annotations
    print(f"\nDetected Text (Brand/Model Info):")
    if texts:
        print(f"Extracted Text: {texts[0].description}")
    else:
        print("No text detected.")


    from math import sqrt
    
    # Define a dictionary of basic color names with their approximate RGB values
    COLOR_NAMES = {
        "Black": (0, 0, 0),
        "White": (255, 255, 255),
        "Red": (255, 0, 0),
        "Lime": (0, 255, 0),
        "Blue": (0, 0, 255),
        "Yellow": (255, 255, 0),
        "Cyan": (0, 255, 255),
        "Magenta": (255, 0, 255),
        "Silver": (192, 192, 192),
        "Gray": (128, 128, 128),
        "Maroon": (128, 0, 0),
        "Olive": (128, 128, 0),
        "Green": (0, 128, 0),
        "Purple": (128, 0, 128),
        "Teal": (0, 128, 128),
        "Navy": (0, 0, 128),
        "Orange": (255, 165, 0),
        "Pink": (255, 192, 203),
        "Brown": (165, 42, 42),
        "Gold": (255, 215, 0)
    }

    def closest_color(requested_color):
        """Find the closest human-readable color name for an RGB value."""
        min_distance = float("inf")
        closest_name = None
        for name, rgb in COLOR_NAMES.items():
            distance = sqrt((rgb[0] - requested_color[0]) ** 2 + 
                            (rgb[1] - requested_color[1]) ** 2 + 
                            (rgb[2] - requested_color[2]) ** 2)
            if distance < min_distance:
                min_distance = distance
                closest_name = name
        return closest_name
    
    # Color Detection
    image_props_response = client.image_properties(image=image)
    props = image_props_response.image_properties_annotation
    
    print("\nDominant Colors:")
    for color in props.dominant_colors.colors[:3]:  # Get top 3 colors
        rgb_value = (int(color.color.red), int(color.color.green), int(color.color.blue))
        color_name = closest_color(rgb_value)  # Convert RGB to color name
        print(f"RGB: {rgb_value} → Color: {color_name} (Confidence: {color.score:.2f})")


# Run detection
detect_image_features('amazon.png')

Detected Labels:
Label: Text (Confidence: 0.95)
Label: Screenshot (Confidence: 0.93)
Label: Number (Confidence: 0.92)
Label: Font (Confidence: 0.90)
Label: Web page (Confidence: 0.87)
Label: Website (Confidence: 0.79)
Label: Software (Confidence: 0.66)
Label: Computer program (Confidence: 0.61)
Label: Sign (Confidence: 0.61)
Label: Document (Confidence: 0.59)

Detected Objects:

Detected Logos:
Logo: Amazon.com (Confidence: 0.72)

Detected Text (Brand/Model Info):
Extracted Text: amazonassociates
Creating Your Amazon Associates Account
English - EN
Hi Geethika k
Account Information
Website and Mobile App List
Profile
Start Using Associates Central
Congrats, Geethika k
Thank you for applying to the Amazon.com Associates Program
Your unique Associate ID is geethikak-20
You may occasionally need to provide your Associate ID to verify your account with us. Your application will be
reviewed shortly after you've referred qualified sales to Amazon.com.
In the meantime, you have been granted t